In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import *
import pyomo.environ as pyo
import pyomo
from math import cos, sin, pi

In [2]:
sheet_name = 'Kotake Mukaihara'
file_path_ratio = 'restaurant_ratios_by_type.xlsx'
df_ratio = pd.read_excel(file_path_ratio, sheet_name="Sheet2", index_col=0).squeeze()
file_path = 'Top10_population_stations.xlsx'
df_station_index = pd.read_excel(file_path, sheet_name=sheet_name, usecols='A:AJ', nrows=1)
df_type = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=13, nrows=12, usecols='B:CJ')

top_stations = df_station_index['Station'].tolist()
nutrition_score = df_station_index['Average_Nutrition_Score'].values[0]
avg_carbon_score = df_station_index['Average_Carbon_Score_Normalized4'].values[0]
avg_price_score = df_station_index['Average_Price_Score'].values[0]
people = df_station_index['S1222_NumberOfPassengers_Cli2.S12_049'].values[0]
df_type_original = df_type.copy(deep=True)
restaurant_types = df_type_original.shape[1]

In [20]:
# Define the objective function components
def diversity(model):
    total_restaurants = sum(int(pyo.value(model.X[i])) for i in range(restaurant_types))
    proportions = [int(pyo.value(model.X[i])) / total_restaurants for i in range(restaurant_types)]
    proportions = np.array(proportions)
    proportions[proportions == 0] = 1  # Prevent taking log(0)
    entropy = -np.sum(proportions * np.log(proportions))
    result = ((entropy * 0.36377347) * (total_restaurants * 0.00043422))
    result = np.power(result, 1/5)
    return result

def crowdedness(model):
    total_restaurants_expr = sum(model.X[i] for i in range(restaurant_types))
    total_restaurants = pyo.value(total_restaurants_expr)
    station_passengers = df_station_index['S1222_NumberOfPassengers_Cli2.S12_049'].iloc[0]
    if station_passengers == 0:
        result = float('inf') 
    else:
        result = ((total_restaurants / station_passengers) * 2.0279 - 0.0001) ** (1/12)
    return result

def total_restaurants_expr(model):
    return sum(model.X[i] for i in range(restaurant_types))

def calculate_area(diversity_val, crowdedness_val, Nutrition_Score, Price_Score, Carbon_Score):
    angles = np.linspace(0, 2 * np.pi, 5, endpoint=False)
    values = [diversity_val, crowdedness_val, Nutrition_Score, Price_Score, Carbon_Score]
    coordinates = [(v * np.cos(a), v * np.sin(a)) for v, a in zip(values, angles)]
    def shoelace(coords):
        x, y = zip(*coords)
        return 0.5 * abs(sum(x[i] * y[i + 1] - x[i + 1] * y[i] for i in range(-1, len(coords) - 1)))
    return shoelace(coordinates)

def calculate_average_area(df, diversity_val, crowdedness_val, price_score_val):
    avg_areas = []
    for index, row in df.iterrows():
        Nutrition_Score = row['Average_Nutrition_Score']
        Price_Score = row['Average_Price_Score']
        Carbon_Score = row['Average_Carbon_Score_Normalized4']
        area = calculate_area(diversity_val, crowdedness_val, Nutrition_Score, price_score_val, Carbon_Score)
        avg_areas.append(area)
    return np.mean(avg_areas)

# Assuming restaurant_types and df_station_index are defined somewhere globally
df_ratio = df_ratio # Ensure this scaling makes sense in your context

# Define the model
model = pyo.ConcreteModel()

# Define the variables
model.X = pyo.Var(range(restaurant_types), domain=pyo.NonNegativeReals, initialize=lambda model, i: df_type_original.values[0][i])
model.price_layer = pyo.Var(range(restaurant_types), domain=pyo.NonNegativeReals, bounds=(0, 1), initialize=0.5)
model.diversity = pyo.Var(initialize=0)
model.crowdedness = pyo.Var(initialize=0)
model.avg_area = pyo.Var(initialize=0)
model.price_score = pyo.Var(initialize=0)

# Define the constraints
model.sum_constraint = pyo.Constraint(expr=sum(model.X[i] for i in range(restaurant_types)) >= 78)

model.initial_constraints = pyo.ConstraintList()
for i in range(restaurant_types):
    model.initial_constraints.add(model.X[i] >= df_type_original.values[0][i])

total_initial_restaurants = sum(df_type_original.values[0][i] for i in range(restaurant_types))

model.ratio_constraints = pyo.ConstraintList()
for i in range(restaurant_types):
    total_restaurants_expr = sum(model.X[j] for j in range(restaurant_types))
    model.ratio_constraints.add((0 <= model.X[i] / total_restaurants_expr) <= df_ratio[i])

def total_restaurants(model):
    return sum(model.X[i] for i in range(restaurant_types))

# Add constraints based on station passenger flow
average_ratio = 0.001686816
station_passengers = df_station_index['S1222_NumberOfPassengers_Cli2.S12_049'].iloc[0]
# total_restaurants = sum(int(pyo.value(model.X[i])) for i in range(restaurant_types))
model.passenger_flow_constraints = pyo.Constraint(expr=total_restaurants(model) <= station_passengers * average_ratio)

# # Price layer constraints
# model.price_layer_constraints = pyo.ConstraintList()
# for i in range(restaurant_types):
#     model.price_layer_constraints.add(model.price_layer[i] <= 0.547957)
#     model.price_layer_constraints.add(model.price_layer[i] >= 0.5)

# Define the objective: minimize total restaurants deviation from initial value
initial_restaurants = sum(df_type_original.values[0][i] for i in range(restaurant_types))
model.objective = pyo.Objective(expr=total_restaurants_expr(model) - initial_restaurants, sense=pyo.minimize)
# model.objective = pyo.Objective(expr=model.avg_area - 0.77, sense=pyo.minimize)

# Define expressions for diversity and crowdedness
model.diversity_expr = pyo.Constraint(expr=model.diversity == diversity(model))
model.crowdedness_expr = pyo.Constraint(expr=model.crowdedness == crowdedness(model))

# Define the new price score without using `if`
def price_score_expr(model):
    total_price_score = sum(model.X[i] * model.price_layer[i] for i in range(restaurant_types))
    total_restaurants = sum(model.X[i] for i in range(restaurant_types))
    return total_price_score / (total_restaurants + 1e-6)  # Adding a small number to avoid division by zero

model.price_score_expr = pyo.Expression(rule=price_score_expr)

# Update the constraint to use the expression
def avg_area_constraint(model):
    diversity_val = diversity(model)
    crowdedness_val = crowdedness(model)
    price_score_val = model.price_score_expr
    avg_area = calculate_average_area(df_station_index, diversity_val, crowdedness_val, price_score_val)
    return avg_area >= 0.77

model.avg_area_constraint = pyo.Constraint(rule=avg_area_constraint)
# Define additional constraints
model.total_price_score_constraint = pyo.Constraint(expr=model.price_score_expr <= 0.547957217)#参数选择的依据：东京所有车站的平均值

# Define the solver
solver = pyo.SolverFactory('ipopt')

# Solve the problem
results = solver.solve(model, tee=True)

# Recompute the diversity and crowdedness values after optimization
diversity_val = diversity(model)
crowdedness_val = crowdedness(model)
price_score_val = pyo.value(model.price_score_expr)

# Calculate the final area
final_area = calculate_average_area(df_station_index, diversity_val, crowdedness_val, price_score_val)

# Display the results
model.display()

# Check if constraints are satisfied
constraints_satisfied = True
for constr in model.component_objects(pyo.Constraint, active=True):
    for index in constr:
        body_value = pyo.value(constr[index].body)
        if constr[index].has_ub():
            satisfied_upper = body_value <= constr[index].upper()
        else:
            satisfied_upper = True
        if constr[index].has_lb():
            satisfied_lower = body_value >= constr[index].lower()
        else:
            satisfied_lower = True
        constraints_satisfied = constraints_satisfied and satisfied_upper and satisfied_lower

print("Constraints satisfied:", constraints_satisfied)
#
    
# Write results to Excel
decision_variables = {f'X_{i}': [round(pyo.value(model.X[i]))] for i in range(restaurant_types)}
price_layers = {f'Price_Layer_{i}': [round(pyo.value(model.price_layer[i]), 2)] for i in range(restaurant_types)}

df_decision_variables = pd.DataFrame(decision_variables)
df_price_layers = pd.DataFrame(price_layers)

# Calculate the parameters for the average area calculation
avg_areas_data = []
for index, row in df_station_index.iterrows():
    Nutrition_Score = row['Average_Nutrition_Score']
    Price_Score = row['Average_Price_Score']
    Carbon_Score = row['Average_Carbon_Score_Normalized4']
    avg_areas_data.append([diversity_val, crowdedness_val, Nutrition_Score, price_score_val, Carbon_Score])
df_avg_areas = pd.DataFrame(avg_areas_data, columns=['Diversity', 'Crowdedness', 'Nutrition_Score', 'Price_Score', 'Carbon_Score'])

# Add the final area calculation to the output
df_final_area = pd.DataFrame({'Final_Area': [final_area]})

with pd.ExcelWriter('Price_optimization_results_Kotake_Mukaihara.xlsx') as writer:
    df_decision_variables.to_excel(writer, sheet_name='Decision_Variables', index=False)
    df_price_layers.to_excel(writer, sheet_name='Price_Layers', index=False)
    df_avg_areas.to_excel(writer, sheet_name='Average_Areas', index=False)
    df_final_area.to_excel(writer, sheet_name='Final_Area', index=False)

Ipopt 3.14.16: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:     8178
Number of nonzeros in Lagrangian Hessian.............:    15225

Total number of variables............................:      176
                     variables with only lower bounds:       87
                variables with lower and upper bounds:       87
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number